In [8]:
# Imports
import pandas as pd
import datetime as dt
import json
import requests
import os
from dotenv import load_dotenv
from datetime import datetime

In [ ]:
## Steps
"""
1. Connecting to CoinAPI using Python
2. Getting the historical data of a cryptocurrency
3. Renaming, dropping and reordering columns from the data we retrieve
4. Using DateTime to get the day of the week and store this information as a new column
5. Saving data from the API to a CSV file
6. Taking the information for a CSV file into a Pandas DateFrame
7. Analysing the data to find things such as the mean, median, percentiles and more
8. Filtering the data by different dates
"""

In [28]:
#Retrive environment variable and store in Python variable
load_dotenv()
api_key= os.getenv('COIN_API_KEY')
# generate url to retrive data
start_date = datetime.strptime('2015-01-01', '%Y-%m-%d') # get data from the start of 2015
end_date = datetime.now() # up until today
start_date = start_date.strftime("%Y-%m-%dT%H:%M:%S")
end_date = end_date.strftime("%Y-%m-%dT%H:%M:%S")
url = "https://rest.coinapi.io/v1/ohlcv/LTC/GBP/history?period_id=1DAY&time_start={}&time_end={}&limit=100000"
json_url=url.format(start_date, end_date)
headers = {"X-CoinAPI-Key" : api_key}
response = requests.get(json_url, headers = headers)

if(response.status_code == 429):
    # check API response
    print("Too many requests.")
#processing and storing data
coin_data  = response.json()

https://rest.coinapi.io/v1/ohlcv/LTC/GBP/history?period_id=1DAY&time_start=2015-01-01T00:00:00&time_end=2020-12-18T17:22:08&limit=100000
2015-01-01T00:00:00
2020-12-18T17:22:08
[{'time_period_start': '2018-09-07T00:00:00.0000000Z', 'time_period_end': '2018-09-08T00:00:00.0000000Z', 'time_open': '2018-09-07T13:02:13.7590000Z', 'time_close': '2018-09-07T18:52:34.5660000Z', 'price_open': 43.92, 'price_high': 43.99, 'price_low': 43.2, 'price_close': 43.57, 'volume_traded': 52.81556576, 'trades_count': 6}, {'time_period_start': '2018-09-08T00:00:00.0000000Z', 'time_period_end': '2018-09-09T00:00:00.0000000Z', 'time_open': '2018-09-08T00:16:46.6120000Z', 'time_close': '2018-09-08T23:42:08.8490000Z', 'price_open': 45.0, 'price_high': 47.68, 'price_low': 41.39, 'price_close': 44.99, 'volume_traded': 128.97727813, 'trades_count': 59}, {'time_period_start': '2018-09-09T00:00:00.0000000Z', 'time_period_end': '2018-09-10T00:00:00.0000000Z', 'time_open': '2018-09-09T00:40:45.4710000Z', 'time_close'

## Define DataFrame

In [29]:
#creating dataframe
ltc_data = pd.DataFrame(coin_data)
#renaming column
ltc_data.rename(columns = {
    "time_period_start": "Start Time",
    "time_period_end": "End Time",
    "time_open": "Open Time",
    "time_close": "Close Time",
    "price_open": "Price Open",
    "price_high": "Price High",
    "price_low": "Price Low",
    "price_close": "Price Close",
    "volume_traded": "Volume Traded",
    "trades_count": "Trade Count",
}, inplace = True)
#dropping columns
ltc_data.drop(["End Time", "Open Time", "Close Time"], axis = "columns", inplace = True)
#reorder columns
reorder_columns = [
    'Start Time',
    'Price Open',
    'Price Close',
    'Price High',
    'Price Low',
    'Volume Traded',
    'Trade Count'
]
ltc_data = ltc_data.reindex(columns = reorder_columns)
#getting days of the week from the date
ltc_data["Start Time"] = pd.to_datetime(ltc_data["Start Time"])
ltc_data["Day of the Week"] = ltc_data['Start Time'].dt.dayofweek

## Define Functions

In [30]:
def number_to_day(number):
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    return days[number]

In [31]:
def saveto_csv(dataFrame,fileName):    
    newfilename= "Data\\"  + fileName +".csv"    
    dataFrame.to_csv(newfilename,index = True)

In [32]:
ltc_data["Day of the Week"] = ltc_data["Day of the Week"].apply(number_to_day)
ltc_data.to_csv("LTC Day History.csv", index = False)